# BoardGameGeeks Dataset

Retrieve data using the BGG API and create an XML file using the following cells:

In [ ]:
%pip install requests
%pip install lxml

In [2]:
import requests
import time
import xml.etree.ElementTree as ET

In [ ]:
# Function to send a request and return XML content
def fetch_game_data(batch_ids):
    ids_string = ",".join(map(str, batch_ids))
    detailed_url = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ids_string}&type=boardgame&comments=1&stats=1"

    try:
        response = requests.get(detailed_url)
        
        if response.status_code == 200:
            return response.content  # Return the raw XML content
        else:
            print(f"Failed to fetch data for games {batch_ids}. Status Code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# Define game ids that should be retrieved (max id for boardgame is 124172)
game_ids = list(range(1,124173))  

In [ ]:
game_ids_sub = game_ids  # Copy original list for processing
batch_size = 20  # Number of games to request per API call
all_items = []

# Process game IDs in batches to avoid request limits
for i in range(0, len(game_ids_sub), batch_size):
    batch = game_ids_sub[i:i + batch_size]
    xml_data = fetch_game_data(batch)
    
    if xml_data:
        root = ET.fromstring(xml_data)
        all_items.extend(root.findall("item"))  # Collect all <item> elements
    
    time.sleep(5)  # Pause between requests to avoid HTTP 429 errors

# Create a new XML tree with a single <items> root
final_root = ET.Element("items")
for item in all_items:
    final_root.append(item)

# Write the final combined XML file
final_tree = ET.ElementTree(final_root)
with open("all_bgg_games.xml", "wb") as file:
    file.write(b"<?xml version='1.0' encoding='UTF-8'?>\n")
    final_tree.write(file, encoding="utf-8", xml_declaration=False)

print("All game data fetched and saved successfully!")

Test Prints to view how many items were written to the file:

In [ ]:
# To access specific elements from the XML file
import xml.etree.ElementTree as ET

# Load and parse the XML
tree = ET.parse("all_bgg_games.xml")
root = tree.getroot()

# Get all <item> elements
items = root.findall("item")

# Count the number of <item> elements
item_count = len(root.findall("item"))

print(f"Total number of board games in the file: {item_count}")

# Get the last 9 items
last_items = items[-9:]

# Print details for each
for i, item in enumerate(last_items, start=1):
    game_id = item.attrib.get("id", "N/A")
    name_elem = item.find("name")
    name = name_elem.attrib.get("value") if name_elem is not None else "Unknown"

    print(f"{i}. ID: {game_id}, Name: {name}")


Total number of board games in the file: 53002
1. ID: 126981, Name: Sixth Tackle
2. ID: 126982, Name: Qwikscore Soccer
3. ID: 126983, Name: Rugby League Trio's
4. ID: 126984, Name: Qwikscore Hockey
5. ID: 126990, Name: Mafioso: The Underworld Strategy Card Game
6. ID: 126994, Name: BASSMASTER Fishing Trivia Game
7. ID: 126996, Name: King's Forge
8. ID: 126997, Name: Oldies Game
9. ID: 126999, Name: Coco Capitano


### Import data into SQL DB

In [ ]:
%pip install psycopg2

Run the following cells after creating a database in PostgreSQL. Use the build_schema.sql file to create the necessary tables. Update the values in the cell below if necessary.

In [ ]:
import xml.etree.ElementTree as ET
import psycopg2

DB_HOST = "localhost"
DB_NAME = "bgg_games"
DB_USER = "postgres"
DB_PASSWORD = "postgres"

xml_data = "all_bgg_games.xml"

In [ ]:
# Functions to insert the data into the database
def insert_auxiliary_entity(cursor, table_name, id_col, name_col, entity_id, entity_name):
    query = f"""
        INSERT INTO {table_name} ({id_col}, {name_col})
        VALUES (%s, %s)
        ON CONFLICT ({id_col}) DO NOTHING;
    """
    cursor.execute(query, (entity_id, entity_name))

def insert_bridge_table(cursor, table_name, game_id, entity_id, game_id_col="game_id", entity_id_col=None):
    if entity_id_col is None:
        entity_id_col = table_name.split('_')[-1] + "_id"
    query = f"""
        INSERT INTO {table_name} ({game_id_col}, {entity_id_col})
        VALUES (%s, %s)
        ON CONFLICT ({game_id_col}, {entity_id_col}) DO NOTHING;
    """
    cursor.execute(query, (game_id, entity_id))

def insert_boardgame(cursor, game):
    game_id = int(game.attrib["id"])
    thumbnail = game.findtext("thumbnail")
    image = game.findtext("image")
    primary_name = None
    for name in game.findall("name"):
        if name.attrib.get("type") == "primary":
            primary_name = name.attrib.get("value")
            break
    if primary_name is None:
        primary_name = "Unknown"

    description = game.findtext("description")
    year_published = game.find("yearpublished")
    year_published = int(year_published.attrib["value"]) if year_published is not None else None

    min_players = game.find("minplayers")
    min_players = int(min_players.attrib["value"]) if min_players is not None else None

    max_players = game.find("maxplayers")
    max_players = int(max_players.attrib["value"]) if max_players is not None else None

    playing_time = game.find("playingtime")
    playing_time = int(playing_time.attrib["value"]) if playing_time is not None else None

    min_playing_time = game.find("minplaytime")
    min_playing_time = int(min_playing_time.attrib["value"]) if min_playing_time is not None else None

    max_playing_time = game.find("maxplaytime")
    max_playing_time = int(max_playing_time.attrib["value"]) if max_playing_time is not None else None

    min_age = game.find("minage")
    min_age = int(min_age.attrib["value"]) if min_age is not None else None

    query = """
        INSERT INTO boardgames (game_id, thumbnail, image, primary_name, description,
            year_published, min_players, max_players, playing_time, min_playing_time,
            max_playing_time, min_age)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (game_id) DO NOTHING;
    """
    cursor.execute(query, (game_id, thumbnail, image, primary_name, description,
                           year_published, min_players, max_players, playing_time,
                           min_playing_time, max_playing_time, min_age))

def insert_alternate_names(cursor, game):
    game_id = int(game.attrib["id"])
    for name in game.findall("name"):
        if name.attrib.get("type") != "primary":
            alt_name = name.attrib.get("value")
            query = """
                INSERT INTO alternate_names (game_id, alternate_name)
                VALUES (%s, %s)
                ON CONFLICT (game_id, alternate_name) DO NOTHING;
            """
            cursor.execute(query, (game_id, alt_name))

def insert_boardgame_stats(cursor, game):
    game_id = int(game.attrib["id"])
    stats = game.find("statistics/ratings")
    if stats is not None:
        def get_int(tag):
            elem = stats.find(tag)
            if elem is not None:
                try:
                    return int(elem.attrib.get("value"))
                except ValueError:
                    return None
            return None

        def get_float(tag):
            elem = stats.find(tag)
            if elem is not None:
                try:
                    return float(elem.attrib.get("value"))
                except ValueError:
                    return None
            return None

        # # Debug helper to list found tags
        # for tag in stats:
        #     print(f"Found tag: {tag.tag}, attribs: {tag.attrib}")

        users_rated = get_int("usersrated")
        average = get_float("average")
        bayes_average = get_float("bayesaverage")
        stddev = get_float("stddev")
        median = get_int("median")
        owned = get_int("owned")
        trading = get_int("trading")
        wanting = get_int("wanting")
        wishing = get_int("wishing")
        num_comments = get_int("numcomments")
        num_weights = get_int("numweights")
        average_weight = get_float("averageweight")

        query = """
            INSERT INTO boardgame_stats (
                game_id, users_rated, average, bayes_average, stddev,
                median, owned, trading, wanting, wishing,
                num_comments, num_weights, average_weight
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (game_id) DO NOTHING;
        """
        cursor.execute(query, (
            game_id, users_rated, average, bayes_average, stddev,
            median, owned, trading, wanting, wishing,
            num_comments, num_weights, average_weight
        ))



def insert_boardgame_ranks(cursor, game):
    game_id = int(game.attrib["id"])
    ranks = game.findall("statistics/ratings/ranks/rank")
    for rank in ranks:
        rank_type = rank.attrib.get("type")
        rank_id = int(rank.attrib.get("id"))
        name = rank.attrib.get("name")
        friendly_name = rank.attrib.get("friendlyname")
        
        # Handle non-numeric 'value'
        value = rank.attrib.get("value")
        try:
            value = int(value)
        except (ValueError, TypeError):
            value = None

        # Handle non-numeric 'bayesaverage'
        try:
            bayes_average = float(rank.attrib["bayesaverage"])
        except (ValueError, KeyError):
            bayes_average = None

        query = """
            INSERT INTO boardgame_ranks (game_id, rank_type, rank_id, name, friendly_name, value, bayes_average)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (game_id, rank_id) DO NOTHING;
        """
        cursor.execute(query, (game_id, rank_type, rank_id, name, friendly_name, value, bayes_average))

def insert_polls_and_options(cursor, game):
    game_id = int(game.attrib["id"])
    
    # Look for poll-summary section
    bestwith = None
    recommendedwith = None
    for summary in game.findall("poll-summary"):
        if summary.attrib.get("name") == "suggested_numplayers":
            for result in summary.findall("result"):
                result_name = result.attrib.get("name")
                result_value = result.attrib.get("value")
                if result_name == "bestwith":
                    bestwith = result_value
                elif result_name in ("recommendedwith", "recommmendedwith"):
                    recommendedwith = result_value

    # Insert all polls (same as before)
    polls = game.findall("poll")
    for poll in polls:
        poll_name = poll.attrib.get("name")
        title = poll.attrib.get("title")
        total_votes = int(poll.attrib.get("totalvotes", 0))

        cursor.execute("""
            INSERT INTO polls (game_id, name, title, total_votes, bestwith, recommendedwith)
            VALUES (%s, %s, %s, %s, %s, %s) RETURNING poll_id
        """, (game_id, poll_name, title, total_votes, bestwith, recommendedwith))
        poll_id = cursor.fetchone()[0]

        for results in poll.findall("results"):
            category = results.attrib.get("numplayers", "Not Applicable")
            for result in results.findall("result"):
                target = result.attrib.get("value")
                value = result.text
                cursor.execute("""
                    INSERT INTO poll_options (poll_id, target, category, value)
                    VALUES (%s, %s, %s, %s)
                    ON CONFLICT (poll_id, target, category) DO NOTHING;
                """, (poll_id, target, category, value))


from psycopg2.extras import execute_values

def insert_user_comments(cursor, game):
    game_id = int(game.attrib["id"])
    comments = game.findall("comments/comment")
    
    values = []
    for comment in comments:
        username = comment.attrib.get("username")
        rating_str = comment.attrib.get("rating")
        try:
            rating = float(rating_str) if rating_str not in (None, '', 'N/A') else None
        except ValueError:
            rating = None
        comment_text = comment.attrib.get("value")
        values.append((game_id, username, rating, comment_text))
    
    if values:
        query = """
            INSERT INTO user_comments (game_id, username, rating, comment)
            VALUES %s;
        """
        execute_values(cursor, query, values)


def insert_auxiliary_entities_and_bridges(cursor, game):
    game_id = int(game.attrib["id"])

    # Helper to insert any link type
    def process_link(type_name, aux_table, aux_id_col, aux_name_col, bridge_table):
        for elem in game.findall(f"link[@type='{type_name}']"):
            entity_id = int(elem.attrib["id"])
            entity_name = elem.attrib["value"]
            insert_auxiliary_entity(cursor, aux_table, aux_id_col, aux_name_col, entity_id, entity_name)
            insert_bridge_table(cursor, bridge_table, game_id, entity_id, entity_id_col=aux_id_col)

    process_link("boardgamepublisher", "publishers", "publisher_id", "publisher_name", "boardgame_publishers")
    process_link("boardgamedesigner", "designers", "designer_id", "designer_name", "boardgame_designers")
    process_link("boardgameartist", "artists", "artist_id", "artist_name", "boardgame_artists")
    process_link("boardgamecategory", "categories", "category_id", "category_name", "boardgame_categories")
    process_link("boardgamemechanic", "mechanics", "mechanic_id", "mechanic_name", "boardgame_mechanics")
    process_link("boardgamefamily", "families", "family_id", "family_name", "boardgame_families")
    process_link("boardgameexpansion", "expansions", "expansion_id", "expansion_name", "boardgame_expansions")
    process_link("boardgameimplementation", "implementations", "implementation_id", "implementation_name", "boardgame_implementations")
    process_link("boardgameintegration", "integrations", "integration_id", "integration_name", "boardgame_integrations")
    process_link("boardgameaccessory", "accessories", "accessory_id", "accessory_name", "boardgame_accessories")
    process_link("boardgamecompilation", "compilations", "compilation_id", "compilation_name", "boardgame_compilations")


In [ ]:
def main():
    tree = ET.parse(xml_data)
    root = tree.getroot()

    conn = psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cursor = conn.cursor()
    print("Connected to DB...")

    counter = 0

    for game in root.findall("item"):
        try:
            insert_boardgame(cursor, game)
            insert_alternate_names(cursor, game)
            insert_boardgame_stats(cursor, game)
            insert_boardgame_ranks(cursor, game)
            insert_polls_and_options(cursor, game)
            insert_user_comments(cursor, game)  # takes the longest of the tables. Consider commenting this line out and running it separately.
            insert_auxiliary_entities_and_bridges(cursor, game)

            counter += 1
            print(f"Progress: Game {counter}/53002")

        except Exception as e:
            print(f"Error processing game_id={game.attrib.get('id')}: {e}")
            conn.rollback()
        else:
            conn.commit()

    cursor.close()
    conn.close()

if __name__ == "__main__":
    main()
    print("Data importing done!")

Data importing done!
